In [1]:

import sys
import os

print("python path", sys.executable)
print("python version", sys.version)

richrdf_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(richrdf_path)

try:
    import ROOT
except ImportError:
    import richrdf.experimental.setupROOT
    cvmfsversion="2024-10-03/x86_64-almalinux9-gcc14.2.0-opt"
    print(f"\nNo ROOT can be imported, trying to set up ROOT from cvmfs {cvmfsversion}\n")
    try:
        import richrdf.experimental
        richrdf.experimental.setupROOT.setupROOT(cvmfsversion=cvmfsversion)
        import ROOT
        print("\nROOT from cvmfs loaded\n")
    except ImportError:
        print("\nROOT not found, please set up ROOT\n")

import richrdf
print(richrdf.get_include_path())



python path /home/llr/ilc/liang/data/liang/software/anaconda3/envs/anaedm/bin/python
python version 3.11.9 | packaged by conda-forge | (main, Apr 19 2024, 18:36:13) [GCC 12.3.0]

No ROOT can be imported, trying to set up ROOT from cvmfs 2024-10-03/x86_64-almalinux9-gcc14.2.0-opt

CVMFS version 2024-10-03/x86_64-almalinux9-gcc14.2.0-opt
LDDIR  ['/cvmfs/sw.hsf.org/key4hep/releases/2024-10-03/x86_64-almalinux9-gcc14.2.0-opt/root/6.32.04-vms5ij/lib', '/cvmfs/sw.hsf.org/key4hep/releases/2024-10-03/x86_64-almalinux9-gcc14.2.0-opt/edm4hep/0.99.1-zincmm/lib', '/cvmfs/sw.hsf.org/key4hep/releases/2024-10-03/x86_64-almalinux9-gcc14.2.0-opt/podio/1.1-ur2cnz/lib']
ROOTDIR ['/cvmfs/sw.hsf.org/key4hep/releases/2024-10-03/x86_64-almalinux9-gcc14.2.0-opt/root/6.32.04-vms5ij/lib']
Includes ['/cvmfs/sw.hsf.org/key4hep/releases/2024-10-03/x86_64-almalinux9-gcc14.2.0-opt/edm4hep/0.99.1-zincmm/include', '/cvmfs/sw.hsf.org/key4hep/releases/2024-10-03/x86_64-almalinux9-gcc14.2.0-opt/podio/1.1-ur2cnz/include']

In [2]:
if not os.path.exists("zh_mumu_REC.edm4hep.root"):
    # Download the sample file
    import urllib.request
    url = "https://raw.githubusercontent.com/lhprojects/edm4hep_sample/refs/heads/main/ILD/zh_mumu_REC.edm4hep.root"
    urllib.request.urlretrieve(url, "zh_mumu_REC.edm4hep.root")
    


In [3]:
import richrdf
ROOT.gInterpreter.AddIncludePath(richrdf.get_include_path())
ROOT.gInterpreter.Declare('#include "analyzeEvent.cpp"')

True

In [4]:
filename = "zh_mumu_REC.edm4hep.root"
df = richrdf.RichRDF("events", filename)
df = df.ReadEvent("event", ["MCParticles", "PandoraPFOs"], throwExceptionOnRefCollIDNotFound=False)
df = df.Define("ans", "analyzeEvent(event)")
df = df.Define("pi0_n", "ans.pi0_E.size()")
df = df.Define("photon_n", "ans.ph_E1.size()")


ph_sum = df.Sum("photon_n")
pi0_sum = df.Sum("pi0_n")
        
ratio = ph_sum.GetValue()/pi0_sum.GetValue()
print("Braching ratios", ratio)

Braching ratios 0.967741935483871
